# Some tests with interpolating integrals

Copyright 2024 Stephan Kulla ("Kulla")

Licensed under the Apache License, Version 2.0 (the "Apache License") and Creative Commons Attribution 4.0 International (the "CC-BY License"). You may choose either of these licenses to govern your use of this project.

You may obtain a copy of the Apache License at:
    http://www.apache.org/licenses/LICENSE-2.0

You may obtain a copy of the CC-BY License at:
    https://creativecommons.org/licenses/by/4.0/

Unless required by applicable law or agreed to in writing, software and content distributed under the Apache License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the Apache License for the specific language governing permissions and limitations under the License.

Under the CC-BY License, you are free to share and adapt the material provided you give appropriate credit, provide a link to the license, and indicate if changes were made. See the CC-BY License for full details.

Third-Party Components and Licenses:
This product also includes third-party components which are distributed under their respective licenses. Please refer to the NOTICE file for details.

## Utility functions and imports

In [1]:
%load_ext jupyter_ai_magics

In [5]:
import sympy as sp

x = sp.symbols("x")
f = sp.sin(x)

## Check whether $\int_a^b\dots\int f(x) \mathrm dx = \int_a^b (b-x)^n \mathrm dx$

Compute antiderivative via $\int_a^x f(y) \mathrm dy$:

In [10]:
def antiderivative(f, a):
    y = sp.symbols("y")
    return sp.integrate(f.subs(x, y), (y, a, x))

print("Antiderivative of sin(x):")

antiderivative(sp.sin(x), -1)

Antiderivative of sin(x):


-cos(x) + cos(1)

In [12]:
print(antiderivative(sp.sin(x), -1) == -sp.cos(x)+sp.cos(-1))
print(antiderivative(sp.exp(x), 4) == sp.exp(x)-sp.exp(4))

True
True


n-th time derivative $\int\dots\int f(y) \mathrm dy$:

In [63]:
def nth_antiderivative(n, f, a):
    for i in range(n+1):
        f = antiderivative(f, a)

    return f

print("5. antiderivative of sin(x):")
print(nth_antiderivative(2, sp.sin(x), -1))

print("Tests:")
print(nth_antiderivative(0, sp.sin(x), -1) == antiderivative(sp.sin(x), -1))
print(nth_antiderivative(1, sp.sin(x), -1) == antiderivative(antiderivative(sp.sin(x), -1), -1))
print(nth_antiderivative(2, sp.sin(x), -1) == antiderivative(antiderivative(antiderivative(sp.sin(x), -1), -1), -1))

print("Test via 1:")
a = sp.symbols("a")
print(nth_antiderivative(0, sp.Integer(1), a) == x-a)
print(nth_antiderivative(1, sp.Integer(1), a) == sp.expand((x-a)**2/2))
print(nth_antiderivative(2, sp.Integer(1), a) == sp.expand((x-a)**3/6))
print(nth_antiderivative(3, sp.Integer(1), a) == sp.expand((x-a)**4/6/4))

5. antiderivative of sin(x):
x**2*cos(1)/2 - x*sin(1) + x*cos(1) + cos(x) - sin(1) - cos(1)/2
Tests:
True
True
True
Test via 1:
True
True
True
True


In [29]:
def nth_definite_antiderivative(n, f, a, b):
    return nth_antiderivative(n, f, a).subs(x, b)

print(nth_definite_antiderivative(0, sp.sin(x), -1, 1) == -sp.cos(1)+sp.cos(-1))

True


In [30]:
for expr in [sp.sin(x), sp.exp(x)]:
    print("Expression ", expr)

    for n in range(5):
        left = nth_definite_antiderivative(n, expr, -1, 1)
        right = sp.integrate((1-x)**n * expr, (x, -1, 1))
        print(f"n={n}: Is {left} = {right}? {left == right}")

Expression  sin(x)
n=0: Is 0 = 0? True
n=1: Is -2*sin(1) + 2*cos(1) = -2*sin(1) + 2*cos(1)? True
n=2: Is -2*sin(1) + 2*cos(1) = -4*sin(1) + 4*cos(1)? False
n=3: Is -2*cos(1)/3 = -4*cos(1)? False
n=4: Is -4*cos(1)/3 + 2*sin(1)/3 = -32*cos(1) + 16*sin(1)? False
Expression  exp(x)
n=0: Is E - exp(-1) = E - exp(-1)? True
n=1: Is E - 3*exp(-1) = E - 3*exp(-1)? True
n=2: Is E - 5*exp(-1) = -10*exp(-1) + 2*E? False
n=3: Is E - 19*exp(-1)/3 = -38*exp(-1) + 6*E? False
n=4: Is E - 7*exp(-1) = -168*exp(-1) + 24*E? False


Neue Rechnung: $\int_a^x \int_a^y \int_a^z f(s) \mathrm ds = \int_a^x \frac 12 (x-y)^2 f(y) \mathrm dy$

In [33]:
for f in [sp.sin(x), sp.exp(x)]:
    print(nth_definite_antiderivative(2, f, -1, 1) == sp.integrate(f / 2 * (1-x)**2,(x, -1, 1)))

True
True


In [44]:
x,y,z = sp.symbols("x y z")
sp.simplify(sp.integrate((y-z)**2/2, (y, z, x)))

x**3/6 - x**2*z/2 + x*z**2/2 - z**3/6

In [43]:
sp.expand((x-z)**3/6)

x**3/6 - x**2*z/2 + x*z**2/2 - z**3/6

Neue Rechnung: $\int_a^x \int_a^y \int_a^z \int_a^t f(s) \mathrm ds\, \mathrm dt\, \mathrm dz\,\mathrm dy = \int_a^x \frac 16 (x-y)^3 f(y) \mathrm dy$

In [65]:
for f in [sp.sin(x), sp.exp(x)]:
    left = nth_definite_antiderivative(3, f, -1, 1)
    right = sp.integrate(f * sp.expand((1-x)**3/6),(x, -1, 1))
    print(left)
    print(right)
    print(left == right)
    print()

-2*cos(1)/3
-2*cos(1)/3
True

E - 19*exp(-1)/3
E - 19*exp(-1)/3
True



In [66]:
for f in [sp.sin(x), sp.exp(x)]:
    left = nth_definite_antiderivative(4, f, -1, 1)
    right = sp.integrate(f * (1-x)**4/24,(x, -1, 1))
    print(left)
    print(right)
    print(left == right)
    print()

-4*cos(1)/3 + 2*sin(1)/3
-4*cos(1)/3 + 2*sin(1)/3
True

E - 7*exp(-1)
E - 7*exp(-1)
True

